<a href="https://colab.research.google.com/github/EhsanHonarvar/Practices/blob/main/1_Bert_StepByStep(Yelp_Review_%D9%8DSeqLen20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re

In [4]:
# full_data = pd.read_csv("/content/drive/My Drive/DataSets/Yelp/train.csv", names= ['Pos_Neg','id','date','query_string','user','Content'] , encoding = 'latin-1')
full_data = pd.read_csv("/content/drive/MyDrive/Datasets/Yelp/train.csv", names= ['Pos_Neg','Content'] , header=None ,encoding = 'latin-1')
full_n = full_data.shape[0]

In [5]:
print(full_n)

560000


In [6]:
print(full_data.groupby(full_data.Pos_Neg).count())

         Content
Pos_Neg         
1         280000
2         280000


In [7]:
print(type(full_data))
full_data.head(5)

<class 'pandas.core.frame.DataFrame'>


,Pos_Neg,Content
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [8]:
import sklearn
from sklearn.model_selection import train_test_split

In [9]:
print(type(full_data))

<class 'pandas.core.frame.DataFrame'>


In [10]:
train_data, dev_data = sklearn.model_selection.train_test_split(full_data, test_size = 0.04) #It was 0.04

In [11]:
train_data.groupby(train_data.Pos_Neg).count()

,Content
Pos_Neg,
1,268879
2,268721


In [ ]:
# train_data = train_data[:160000]
# dev_data = dev_data[:64000]

In [12]:
print(full_data.shape[0],'\n',train_data.shape[0],'\n',dev_data.shape[0])

560000 
 537600 
 22400


In [13]:
train_data.Pos_Neg[0:10]

266777    2
45544     1
507003    2
91766     2
244561    2
397111    2
427504    2
109884    1
201505    2
34998     2
Name: Pos_Neg, dtype: int64

In [14]:
# Get ground truth x and y values
train_x_raw = train_data.loc[:]["Content"]
train_y = [0.0 if y == 2 else 1.0 for y in train_data.loc[:]["Pos_Neg"]]

dev_x_raw = dev_data.loc[:]["Content"]
dev_y = [0.0 if y == 2 else 1.0 for y in dev_data.loc[:]["Pos_Neg"]]
#print (dev_y)

In [15]:
print(train_x_raw[0:3])

266777    This is an awesome restaurant for a bachelor p...
45544     .......Where are the chopsticks?\n\nA Chinese ...
507003    So I'm pretty particular about my boba drinks ...
Name: Content, dtype: object


In [16]:
def clean(data):
  tokens = str(data).split()
  translation_table = str.maketrans('', '', "\"#$%&'()*+-/:;<=>@[\]^_`{|}~?!.,")
  tokens = [w.translate(translation_table) for w in tokens]
  tokens = [word.lower() for word in tokens]

  return ' '.join(tokens)

# data = []
# labels = []



In [17]:
train_x_raw = [clean(y)  for y in train_x_raw]

In [18]:
print(train_x_raw[0:20])

['this is an awesome restaurant for a bachelor party or other large gathering reservations are a breeze with open table the ultimate steak to share with a few people is definitely the budget way to go if you want a steak and a few sides to share family style however our waiter did forget to being the extra side we ordered until we had already finished our meal he was apologetic and did comp us the extra side attentive wait staff and superb presentation ill definitely come here again', 'where are the chopsticksnna chinese restaurant is clearly not chinese when there are no chopsticks on the place setting tea was lukewarm upon arrival tasted unauthentic with the help of any ambiguous american tea bag the hot and sour soup had peaspeas side of rice was stiff and tasted as if it had been in the cooker all day had the stir fry vegetable dish that consisted of broccoli snow peas and mushrooms overly cooked to mush where was my bok choy or kai lan or any type of actual authentic chinese veget

In [19]:
print(train_y[0:20])

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]


In [20]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(train_y)

In [21]:
data = train_x_raw

In [22]:
from transformers import BertTokenizer
import numpy as np

input_ids=[]
attention_masks=[]

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for sent in data:
    bert_inp = bert_tokenizer.encode_plus(sent, add_special_tokens = True, max_length =20, pad_to_max_length = True, # max_length was 64
                                        return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(labels)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
len(input_ids),len(attention_masks),len(labels)


(537600, 537600, 537600)

In [24]:
from sklearn.model_selection import train_test_split

train_inp, val_inp, train_label, val_label, train_mask, val_mask = train_test_split(
   input_ids, labels, attention_masks, test_size=0.20, random_state=1000)

test_inp, val_inp, test_label, val_label, test_mask, val_mask = train_test_split(
   val_inp, val_label, val_mask, test_size=0.5, random_state=1000)


In [25]:

import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 20 # It was 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]
X = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(X)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 20)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 20)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 20,                                            

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 20,
                       validation_data = ([val_inp, val_mask],val_label))

Epoch 1/20


13440/13440 [==============================] - 1913s 139ms/step - loss: 0.3447 - accuracy: 0.8355 - val_loss: 0.3160 - val_accuracy: 0.8532
Epoch 2/20
13440/13440 [==============================] - 1860s 138ms/step - loss: 0.2707 - accuracy: 0.8760 - val_loss: 0.3120 - val_accuracy: 0.8579
Epoch 3/20
13440/13440 [==============================] - 1855s 138ms/step - loss: 0.2092 - accuracy: 0.9075 - val_loss: 0.3363 - val_accuracy: 0.8542
Epoch 4/20
13195/13440 [============================>.] - ETA: 32s - loss: 0.1535 - accuracy: 0.9347

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 20 # It was 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[1]
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(embeddings)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 5,
                       validation_data = ([val_inp, val_mask],val_label))